In [1]:
import torch
import torch.optim as optim
import torch.utils.data
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [50]:
#--- hyperparameters ---
N_EPOCHS = 5
BATCH_SIZE_TRAIN = 100
BATCH_SIZE_TEST = 100
LR = 0.01

In [3]:
#--- fixed constants ---
NUM_CLASSES = 24
DATA_DIR = '../data/sign_mnist_%s'

In [4]:
# --- Dataset initialization ---

# We transform image files' contents to tensors
# Plus, we can add random transformations to the training data if we like
# Think on what kind of transformations may be meaningful for this data.
# Eg., horizontal-flip is definitely a bad idea for sign language data.
# You can use another transformation here if you find a better one.
train_transform = transforms.Compose([
                                        #transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor()])
test_transform = transforms.Compose([transforms.ToTensor()])

train_set = datasets.ImageFolder(DATA_DIR % 'train', transform=train_transform)
dev_set   = datasets.ImageFolder(DATA_DIR % 'dev',   transform=test_transform)
test_set  = datasets.ImageFolder(DATA_DIR % 'test',  transform=test_transform)


# Create Pytorch data loaders
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=BATCH_SIZE_TEST, shuffle=False)

In [48]:
#--- model ---
class CNN(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(CNN, self).__init__()
        # WRITE CODE HERE
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, 5),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(4 * 4 * 32, num_classes)
        

    def forward(self, x):
        # WRITE CODE HERE
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out  = F.softmax(out, dim =1)
        return out

In [51]:
#--- set up ---
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = CNN().to(device)

# WRITE CODE HERE
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
loss_function = nn.CrossEntropyLoss()


#--- training ---
for epoch in range(N_EPOCHS):
    train_loss = 0
    train_correct = 0
    total = 0
    for batch_num, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        
        outputs = model(data)
        total += target.size(0)
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == target).sum().item()
       

        loss=loss_function(outputs, target)
        
        train_loss+=loss
        loss.backward()
        optimizer.step()


        print('Training: Epoch %d - Batch %d/%d: Loss: %.4f | Train Acc: %.3f%% (%d/%d)' % 
              (epoch, batch_num, len(train_loader), train_loss / (batch_num + 1), 
               100. * train_correct / total, train_correct, total))
    
    # WRITE CODE HERE
    # Please implement early stopping here.
    # You can try different versions, simplest way is to calculate the dev error and
    # compare this with the previous dev error, stopping if the error has grown.

 Epoch 4 - Batch 31/275: Loss: 2.5213 | Train Acc: 72.594% (2323/3200)
Training: Epoch 4 - Batch 32/275: Loss: 2.5175 | Train Acc: 73.000% (2409/3300)
Training: Epoch 4 - Batch 33/275: Loss: 2.5173 | Train Acc: 73.029% (2483/3400)
Training: Epoch 4 - Batch 34/275: Loss: 2.5170 | Train Acc: 73.057% (2557/3500)
Training: Epoch 4 - Batch 35/275: Loss: 2.5163 | Train Acc: 73.139% (2633/3600)
Training: Epoch 4 - Batch 36/275: Loss: 2.5178 | Train Acc: 73.000% (2701/3700)
Training: Epoch 4 - Batch 37/275: Loss: 2.5187 | Train Acc: 72.895% (2770/3800)
Training: Epoch 4 - Batch 38/275: Loss: 2.5223 | Train Acc: 72.513% (2828/3900)
Training: Epoch 4 - Batch 39/275: Loss: 2.5220 | Train Acc: 72.575% (2903/4000)
Training: Epoch 4 - Batch 40/275: Loss: 2.5223 | Train Acc: 72.537% (2974/4100)
Training: Epoch 4 - Batch 41/275: Loss: 2.5220 | Train Acc: 72.548% (3047/4200)
Training: Epoch 4 - Batch 42/275: Loss: 2.5240 | Train Acc: 72.349% (3111/4300)
Training: Epoch 4 - Batch 43/275: Loss: 2.5231 | 

In [55]:
#--- test ---
test_loss = 0
test_correct = 0
total = 0

with torch.no_grad():
    for batch_num, (data, target) in enumerate(test_loader):
        data, target = data.to(device), target.to(device)
        # WRITE CODE HERE
        outputs=model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        test_correct += (predicted == target).sum()
        loss=loss_function(outputs, target)
        
        test_loss+=loss
        print('Evaluating: Batch %d/%d: Loss: %.4f | Test Acc: %.3f%% (%d/%d)' % 
              (batch_num, len(test_loader), test_loss / (batch_num + 1), 
               100. * test_correct / total, test_correct, total))


Evaluating: Batch 0/37: Loss: 2.2472 | Test Acc: 100.000% (100/100)
Evaluating: Batch 1/37: Loss: 2.2553 | Test Acc: 99.500% (199/200)
Evaluating: Batch 2/37: Loss: 2.3087 | Test Acc: 93.667% (281/300)
Evaluating: Batch 3/37: Loss: 2.3251 | Test Acc: 92.250% (369/400)
Evaluating: Batch 4/37: Loss: 2.3473 | Test Acc: 90.000% (450/500)
Evaluating: Batch 5/37: Loss: 2.3702 | Test Acc: 87.833% (527/600)
Evaluating: Batch 6/37: Loss: 2.4198 | Test Acc: 82.857% (580/700)
Evaluating: Batch 7/37: Loss: 2.5230 | Test Acc: 72.500% (580/800)
Evaluating: Batch 8/37: Loss: 2.6033 | Test Acc: 64.444% (580/900)
Evaluating: Batch 9/37: Loss: 2.6063 | Test Acc: 64.100% (641/1000)
Evaluating: Batch 10/37: Loss: 2.5874 | Test Acc: 66.000% (726/1100)
Evaluating: Batch 11/37: Loss: 2.5871 | Test Acc: 66.000% (792/1200)
Evaluating: Batch 12/37: Loss: 2.5882 | Test Acc: 65.846% (856/1300)
Evaluating: Batch 13/37: Loss: 2.5907 | Test Acc: 65.571% (918/1400)
Evaluating: Batch 14/37: Loss: 2.5921 | Test Acc: 65